In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from nltk import FreqDist
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer


In [7]:
count_vect = CountVectorizer(min_df=5, stop_words ='english')

help(count_vect.fit_transform)

Help on method fit_transform in module sklearn.feature_extraction.text:

fit_transform(raw_documents, y=None) method of sklearn.feature_extraction.text.CountVectorizer instance
    Learn the vocabulary dictionary and return term-document matrix.
    
    This is equivalent to fit followed by transform, but more efficiently
    implemented.
    
    Parameters
    ----------
    raw_documents : iterable
        An iterable which yields either str, unicode or file objects.
    
    Returns
    -------
    X : array, [n_samples, n_features]
        Document-term matrix.



In [8]:
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
import csv 
import random
import nltk



def reduceText(text, stops):
    text = text.lower() #assign 1
    text = re.sub('\W', ' ', text) #not in set [a-zA-Z0-9_], general purpose to remove punctuation
    tokens = word_tokenize(text)
    return [w for w in tokens if not w in stops]

def get_word_features(docs):
    posWords = []
    negWords = []
    for t, s in docs:
        if s == '1':
            posWords.extend(t)
        else:
            negWords.extend(t)
    word_features = nltk.FreqDist(posWords+negWords)
    return [w for w, n in word_features.most_common()[:2000]]            

def document_features(document, word_features):
    document_words = set(document) 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features


In [101]:
fname = r"reviews_ChinesePN_1000.csv"
f = open(fname, 'r', encoding = 'utf8')
reader = csv.reader(f)

docs = []
target = []

stops = set(stopwords.words("english"))                  
for row in reader:
    break
for row in reader:
    docs.append(row[0])
    target.append(row[1])



collected text and suffled docs


In [102]:
def countRows(fname):
    print("CountRows")
    #fname = "reviews_NEGallUSA" + n + "Res.csv"

    f = open(fname, 'r', encoding = 'utf8')
    
    reader = csv.reader(f)
    x = -1
    for row in reader:
        x+=1

    f.close()  
    return x

In [103]:
countRows('reviews_ChinesePN_1000.csv')

CountRows


1000

In [104]:
len(docs)

1000

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=3, stop_words ='english')


In [106]:
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And the third one.',
     'Is this the first document?']


In [107]:
vectorizer = CountVectorizer(min_df=2)
doc_text = docs
train_data = doc_text[:800]

xtr = vectorizer.fit_transform(doc_text[:800])
xtr = vectorizer.fit_transform(doc_text[800:])

In [108]:
def extract_text_features(train_data, test_data, min_docs ):
    """
    Returns two types of training and test data features.
        1) Bags of words (BOWs): X_train_counts, X_test_counts
        2) Term Frequency times Inverse Document Frequency (tf-idf): X_train_tfidf, X_test_tfidf

    How to create BOW features:
        You need to first generate a count vector from the input data, excluding the NLTK
        stopwords. This can be done by importing the English stopword list from NLTK and then
        passing it to a CountVectorizer during initialization.

        CountVectorizer is slightly different than the FreqDist object you used in your previous
        assignment.  Where FreqDist is good at creating a dict-like bag-of-words representation for
        a single document, CountVectorizer is optimized for creating a sparse matrix representing
        the bag-of-words counts for every document in a corpus of documents all at once.  Both
        objects are useful at different times.

    How to create tf-idf features:
        tf-idf features can be computed using TfidfTransformer with the count matrix (BOWs matrix)
        as an input. The fit method is used to fit a tf-idf estimator to the data, and the
        transform method is used afterwards to transform either the training or test count-matrix
        to a tf-idf representation. The method fit_transform strings these two methods together
        into one.

        For the training data, you'll want to use the fit_transform method to both fit the
        tf-idf model and then transform the training count matrix into a tf-idf representation.

        For the test data, you only need to call the transform method since the tf-idf weights
        will have already been fit on your training set.

    Parameters
    ----------
    train_data : List[str]
        Training News data in list

    test_data : List[str]
        Test data in list
    
    min_docs : integer
        Do not include terms in the vocabulary that occur in less than "min_docs" documents 

    Returns
    -------
    Tuple(scipy.sparse.csr.csr_matrix,..)
        Returns X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf as a tuple.

    Examples
    --------
    >>> X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf =
    ...          extract_text_features(twenty_train.data, sample_test_documents, 1)
    >>> X_train_counts
    <2989x39831 sparse matrix of type '<class 'numpy.int64'>'
	with 377208 stored elements in Compressed Sparse Row format>
    """
 
    from sklearn.feature_extraction.text import TfidfTransformer
    ### YOUR SOLUTION STARTS HERE### 
    
    # Replace FIRSTNAME_LASTNAME with your name
    print(' Student: Janice_Hsu,   Function: extract_text_features()')

    # Generate count vectors from the input data, excluding the NLTK stopwords and
    # ignoring tokens that occur in fewer than "min_docs" documents 
    count_vect = CountVectorizer(min_df=min_docs, stop_words ='english')
    
    # Bags of words (BOWs): X_train_counts, X_test_counts
    X_train_counts = count_vect.fit_transform(train_data) #**SLIGHLTLY DIFFERENT DIM (2989, 3966)
    X_test_counts = count_vect.transform(test_data)

    #X_test_counts = count_vect.transform(test_data)
    
    #Term Frequency times Inverse Document Frequency (tf-idf): X_train_tfidf, X_test_tfidf
    # Compute tfidf feature values and store in 'X_train_tfidf' and 'X_test_tfidf'
    tfidf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    #fit/compute Tfidf weights using X_train_counts
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    #apply fitted weights to X_test_counts
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)


    return (X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf)
    ### END SOLUTIONS ###pass


In [109]:
def fit_and_predict_multinomialNB(X_train, Y_train, X_test):

    """
    Train multinomial naive Bayes model with 'X_train' and 'Y_train' and
    predict the Y values for 'X_test'. (Use 'MultinomialNB' from scikit-learn.)
    Return the predicted Y values.

    Parameters
    ----------
    X_train: scipy sparse matrix
        Data for training (matrix with features, e.g. BOW or tf-idf)
    Y_train: numpy.ndarray
        Labels for training data (target value)
    X_test: scipy sparse matrix
        Test data used for prediction

    Returns
    -------
    numpy.ndarray[int]
        Target values predicted from 'X_test'

 
    """
    # Import the package
    from sklearn.naive_bayes import MultinomialNB 

    ### YOUR SOLUTION STARTS HERE### 
    #used scikit-learn tutorial on training a classifier
    # fit the model... 
    clf = MultinomialNB().fit(X_train, Y_train) #naive bayes
    # make predictions
    predicted_MultinomialnNB = clf.predict(X_test) #predict
    return predicted_MultinomialnNB
    ### END SOLUTION ###  
 


def fit_and_predict_BernoulliNB(X_train, Y_train, X_test):

    """
    Train Bernoulli naive Bayes model with 'X_train' and 'Y_train' and
    predict the Y values for 'X_test'. (Use 'BernoulliNB' from scikit-learn.)
    The 'binarize' threshold should be set to 0.0.
    Return the predicted Y values.


    Parameters
    ----------
    X_train: scipy sparse matrix
        Data for training (matrix with features, e.g. BOW or tf-idf)
    Y_train: numpy.ndarray
        Labels for training data (target value)
    X_test: scipy sparse matrix
        Test data used for prediction

    Returns
    -------
    numpy.ndarray[int]
        Target values predicted from 'X_test'

    """

    # Import the package
    from sklearn.naive_bayes import BernoulliNB 

    ### YOUR SOLUTION STARTS HERE### 
    #referenced to sklearn documentation    
    # fit the model... 
    clf = BernoulliNB(binarize=0.0).fit(X_train, Y_train)  #fit naive bayes to X and Y train data
    # make predictions
    predicted_bernNB = clf.predict(X_test)
    return predicted_bernNB
    ### END SOLUTION ###  


def fit_and_predict_LR(X_train, Y_train, X_test):

    """
    Train logistic regression model with 'X_train' and 'Y_train' and
    predict the Y values for 'X_test'. (Use 'LogisticRegression' from scikit-learn.)
    Return the predicted Y values.


    Parameters
    ----------
    X_train: scipy sparse matrix
        Data for training (matrix with features, e.g. BOW or tf-idf)
    Y_train: numpy.ndarray
        Labels for training data (target value)
    X_test: scipy sparse matrix
        Test data used for prediction

    Returns
    -------
    numpy.ndarray[int]
        Target values predicted from 'X_test'

    """

    # Import the package
    from sklearn.linear_model import LogisticRegression

    ### YOUR SOLUTION STARTS HERE### 
    #referenced to sklearn documentation    
    # Replace FIRSTNAME_LASTNAME with your name
    print(' Student: Janice_Hsu,   Function: fit_and_predict_LR()')
    # fit the model... 
    clf = LogisticRegression().fit(X_train, Y_train)     
    # make predictions 
    predicted_LR = clf.predict(X_test)
    return predicted_LR
    ### END SOLUTION ### 


def fit_and_predict_KNN(X_train, Y_train, X_test, K):

    """
    Train nearest neighbor classifier model with 'X_train' and 'Y_train' and
    predict the Y values for 'X_test'. Use 'KNearestNeighborsClassifier' from 
    scikit-learn with K nearest neighbors (K = 1, 3, 5, ....)
    Return the predicted Y values.


    Parameters
    ----------
    X_train: scipy sparse matrix
        Data for training (matrix with features, e.g. BOW or tf-idf)
    Y_train: numpy.ndarray
        Labels for training data (target value)
    X_test: scipy sparse matrix
        Test data used for prediction
    K: integer (odd)
    	Number of neighbors to use for prediction, e.g., K = 1, 3, 5, ...

    Returns
    -------
    numpy.ndarray[int]
        Target values predicted from 'X_test'

    """
 
    # Import the package
    from sklearn.neighbors import KNeighborsClassifier

    ### YOUR SOLUTION STARTS HERE###
    #referenced to sklearn documentation
    # fit the model (for KNN this is just storing the training data and labels) 
    clf = KNeighborsClassifier(n_neighbors=K).fit(X_train, Y_train)
    # Predict
    predicted_KNN = clf.predict(X_test)
    return predicted_KNN
    ### END SOLUTION  ### 
    

In [110]:
X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf = extract_text_features(docs[:800],  docs[800:],3)
print('Dimensions of X_train_counts are (docsxvocab)',X_train_counts.shape)


 Student: Janice_Hsu,   Function: extract_text_features()
Dimensions of X_train_counts are (docsxvocab) (800, 1863)


In [111]:
train_target = target[:800]
test_target = target[800:]

In [112]:
predicted_multNB = fit_and_predict_multinomialNB(X_train_tfidf, train_target, X_test_tfidf)
predicted_multNB = fit_and_predict_multinomialNB(X_train_tfidf, train_target, X_test_tfidf)
predicted_bernNB = fit_and_predict_BernoulliNB(X_train_tfidf, train_target, X_test_tfidf)
predicted_LR = fit_and_predict_LR(X_train_tfidf, train_target, X_test_tfidf)
predicted_LR = fit_and_predict_LR(X_train_counts, train_target, X_test_counts)
K=3

predicted_KNN = fit_and_predict_KNN(X_train_tfidf, train_target, X_test_tfidf, K)


 Student: Janice_Hsu,   Function: fit_and_predict_LR()
 Student: Janice_Hsu,   Function: fit_and_predict_LR()


In [116]:
sample_test_documents = docs[800:]
print('#####Predicted labels with multinomial NB classifier:') 
for doc, p, i in zip(sample_test_documents, predicted_multNB, range(10)):
    print('{} -> {}, {}'.format(doc[:50], p, test_target[i]))
print()

# # Bernoulli naive Bayes
print('#####Predicted labels with Bernoulli NB classifier:') 
for doc, p, i in zip(sample_test_documents, predicted_bernNB, range(10)):
    print('{} -> {}, {}'.format(doc[:50], p, test_target[i]))
print()


# kNN
print('####Predicted labels with kNN classifier:')
for doc, p, i in zip(sample_test_documents, predicted_KNN, range(10)):
    print('{} -> {}, {}'.format(doc[:50], p, test_target[i]))
print()
#
# Logistic Regression 
print('####Predicted labels with logistic classifier:')
for doc, p, i in zip(sample_test_documents, predicted_LR, range(10)):
    print('{} -> {}, {}'.format(doc[:50], p, test_target[i]))
print()



#####Predicted labels with multinomial NB classifier:
Worst service ever. I sat at the bar.  I ordered b -> 1, 0
Came in for happy hour around 930. Hostess was loo -> 1, 1
My husband and I were frequent customers at PF Cha -> 1, 0
Consistently good. We took advantage of the two 4- -> 1, 1
For what it is, PF Chang's is great. Sorry, Yelp f -> 1, 1
Excellent service - good food.  Although some of o -> 1, 1
I always try with my wife this place with their ch -> 1, 1
I've driven by Chow Mein Express a million times.  -> 1, 0
I've been living in Vegas for quite some time now  -> 1, 1
HORRIBLE!!!TRASH!!!if I could I would give -5 .we  -> 0, 0

#####Predicted labels with Bernoulli NB classifier:
Worst service ever. I sat at the bar.  I ordered b -> 0, 0
Came in for happy hour around 930. Hostess was loo -> 1, 1
My husband and I were frequent customers at PF Cha -> 1, 0
Consistently good. We took advantage of the two 4- -> 1, 1
For what it is, PF Chang's is great. Sorry, Yelp f -> 1, 1
Excellen

In [45]:
def test_classifiers(train, test, min_docs, K):
    #test_classifiers(twenty_train, twenty_test, 1, 3)
    """
    Evaluate the accuracy of multiple classifiers by training on the data in 
    "train" and making predictions on the data in "test". The classifiers
    evaluated are: BernoulliNB, MultinomialNB, Logistic, and kNN.
    
    The input train and test data are scikit-learn objects of type "bunch"
    containing both the raw text as well as label information.
    
    The function first calls extract_text_features() to create a common
    vocabulary and feature set for all the classifiers to use.
    
    The classifiers should use tfidf features.
    
    
    Parameters
    ----------
    train: sklearn.datasets.base.Bunch
        Text data with labels for training each classifier
    test: sklearn.datasets.base.Bunch
        Text data with labels for testing each classifier
    min_docs : integer
        Do not include terms in the vocabulary that occur in less than "min_docs" documents    
    K: integer (odd)
        Number of neighbors to use for prediction, e.g., K = 1, 3, 5, ...
 
    """
    X_train_counts, X_train_tfidf, X_test_counts, X_test_tfidf = extract_text_features(train.data, test.data, min_docs)
    
    num_docs, vocab_size = X_train_counts.shape
    print('Number of documents =',num_docs)
    print('Vocabulary size =',vocab_size)
    

    # Now evaluate the classifiers on the test data
    # Print out the accuracy as a percentage for each classifier.
    # np.mean() can be used to calculate the accuracy. Round the accuracy to 2 decimal places.
    
    import numpy as np
    ### YOUR SOLUTION STARTS HERE###  
    
    #predict according to different classifier--evaluate results
    predicted_multNB = fit_and_predict_multinomialNB(X_train_tfidf, train.target, X_test_tfidf)
    predicted_bernNB = fit_and_predict_BernoulliNB(X_train_tfidf, train.target, X_test_tfidf)
    predicted_LR = fit_and_predict_LR(X_train_tfidf, train.target, X_test_tfidf)
    predicted_KNN = fit_and_predict_KNN(X_train_tfidf, train.target, X_test_tfidf, K)
    
    # count num of correct predictions / total
    multNB = np.sum(predicted_multNB == test.target)/len(test.target) 
    bernNB = np.sum(predicted_bernNB == test.target)/len(test.target)
    LR = np.sum(predicted_LR == test.target)/len(test.target)
    KN = np.sum(predicted_KNN == test.target)/len(test.target)
    
    print('Accuracy with multinomial naive Bayes: {:.3f}'.format(multNB))
    print('Accuracy with Bernoulli naive Bayes: {:.3f}'.format(bernNB))
    print('Accuracy with logistic regression: {:.3f}'.format(LR))
    print('Accuracy with kNN, k={} classifier: {:.3f}'.format(K, KN))

In [53]:
vectorizer.transform(["food was good"]).toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [54]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)

In [55]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)